In [1]:
from ngsolve import *
from netgen.csg import *
import netgen.gui
%gui tk

In [3]:
box = OrthoBrick(Pnt(-3,-3,-3),Pnt(3,3,3)).bc("outer")
magnet = Cylinder(Pnt(-1,0,0),Pnt(1,0,0), 0.3) * OrthoBrick(Pnt(-1,-3,-3),Pnt(1,3,3))
air = box - magnet

geo = CSGeometry()
geo.Add (air.mat("air"))
geo.Add (magnet.mat("magnet").maxh(1))
mesh = Mesh(geo.GenerateMesh(maxh=2))


In [4]:
mesh.GetMaterials(), mesh.GetBoundaries()

(['air', 'magnet'],
 ['outer',
  'outer',
  'outer',
  'outer',
  'outer',
  'outer',
  'default',
  'default',
  'default'])

In [5]:
fes = HCurl(mesh, order=3, dirichlet="outer", flags = { "nograds" : True })
print ("ndof =", fes.ndof)
u = fes.TrialFunction()
v = fes.TestFunction()

from math import pi
mu0 = 4*pi*1e-7
mur = CoefficientFunction( [1000 if mat== "magnet" else 1 
                            for mat in mesh.GetMaterials()]) 

a = BilinearForm(fes)
a += SymbolicBFI(1/(mu0*mur) * curl(u) * curl(v) + 1e-6/(mu0*mur)*u*v)
c = Preconditioner(a, "bddc")

f = LinearForm(fes)
mag = CoefficientFunction((1,0,0))
f += SymbolicLFI(mag*curl(v), definedon=mesh.Materials("magnet"))

ndof = 87940


In [10]:
with TaskManager():
    a.Assemble()
    f.Assemble()

In [11]:
gfu = GridFunction(fes)

inv = CGSolver(a.mat, c.mat)
gfu.vec.data = inv * f.vec
Draw (curl(gfu), mesh, "B-field")

In [9]:
mesh.Curve(3)